In [ ]:
from ratdata import data_manager as dm, process, ingest
from pathlib import Path
import numpy as np

In [ ]:
dm.db_connect('rat_data.db')
outfilename = 'data/teed.csv'
time_slices_file = 'data/mce_recordings/time_slices.pickle'

In [ ]:
# Get recording files from the database
open_loop_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'continuous'))
on_off_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'on-off'))
random_files_query = dm.RecordingFile.select().join(dm.StimSettings).where((dm.StimSettings.stim_type == 'random'))
# Read time slices from the file
time_slices = ingest.read_file_slices(time_slices_file)

In [ ]:
# Calculate TEED for each file with continuous stimulation
with open(outfilename, 'w') as outfile:
    for file in open_loop_files_query:
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        teed = process.compute_teed_continuous_stim(amplitude, pulse_width=80, stim_frequency=128.87, impedance=1)
        line = '%s, %f' % (file.filename, teed) 
        print(line)
        outfile.write(line + '\n')
    

In [ ]:
# Calculate TEED for each file with ON/OFF stim
pulse_width = 80
f_stimulation = 130
with open(outfilename, 'a') as outfile:
    for file in on_off_files_query:
        filename_stem = file.filename.split('.')[0]
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        recording = ingest.read_mce_matlab_file(str(Path(file.dirname) / file.filename))
        amplitude_data = process.generate_amplitude_from_stim_periods(recording, amplitude)
        if filename_stem in time_slices:
            fs = int(1 / recording.dt)
            slice = time_slices[filename_stem]
            amplitude_data, tt = process.trim_recording(amplitude_data, fs, slice['start'], slice['length'])
        else:
            tt = np.linspace(0, recording.electrode_data.shape[1] * recording.dt, recording.electrode_data.shape[1])
        teed = process.compute_teed_from_amplitude_recording(amplitude_data, tt, pulse_width, f_stimulation, 1)
        line = '%s, %f' % (file.filename, teed)
        print(line)
        outfile.write(line + '\n')
    

In [ ]:
# Calculate TEED for each file with random stim
pulse_width = 80
f_stimulation = 130
with open(outfilename, 'a') as outfile:
    for file in random_files_query:
        filename_stem = file.filename.split('.')[0]
        amplitude = dm.StimSettings.get(dm.StimSettings.recording_file == file).max_amplitude
        recording = ingest.read_mce_matlab_file(str(Path(file.dirname) / file.filename))
        amplitude_data = process.generate_amplitude_from_stim_periods(recording, amplitude)
        if filename_stem in time_slices:
            fs = int(1 / recording.dt)
            slice = time_slices[filename_stem]
            amplitude_data, tt = process.trim_recording(amplitude_data, fs, slice['start'], slice['length'])
        else:
            tt = np.linspace(0, recording.electrode_data.shape[1] * recording.dt, recording.electrode_data.shape[1])
        teed = process.compute_teed_from_amplitude_recording(amplitude_data, tt, pulse_width, f_stimulation, 1)
        line = '%s, %f' % (file.filename, teed) 
        print(line)
        outfile.write(line + '\n')

In [ ]:
with open('data/teed.csv', 'r') as f:
    with open('data/teed_full.csv', 'w') as outfile:
        for line in f:
            line = line.strip(' \r\t\n')
            filename, teed = line.split(',')
            record = dm.RecordingFile.select().join(dm.Rat).where(dm.RecordingFile.filename == filename).get()
            stim_settings = dm.StimSettings.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename).get()
            outline = '%s, %s, %s, %s, %s, %s, %s' % (filename, record.condition, record.rat.label, record.rat.group,
                                                    stim_settings.stim_type, stim_settings.max_amplitude, teed)
            print(outline)
            outfile.write(outline + '\n')